In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import pickle

from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scipy.stats import entropy

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir("/content/drive/My Drive/Libin's Project/")
print(os.getcwd())

/content
/content/drive/My Drive/Libin's Project


In [ ]:
class Model(): #tf.Module
  def __init__(self, name=None):

    self.name = name
    self.loss_fn = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    self.train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
    self.opt = tf.optimizers.Adam()

  def build(self):

    self.model =  tf.keras.Sequential()

    self.model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same',strides=1, input_shape=(28,28,1), use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 64, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.Conv2D(128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Conv2D( 128, (3, 3), padding='same',strides=1, use_bias=False))
    self.model.add(tf.keras.layers.BatchNormalization())
    self.model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    self.model.add(tf.keras.layers.Dropout(0.25))

    self.model.add(tf.keras.layers.MaxPooling2D((2,2)))
    self.model.add(tf.keras.layers.Flatten())
    self.model.add(tf.keras.layers.Dense(128))
    self.model.add(tf.keras.layers.Dense(8)) #, activation='softmax'))
    self.model.add(Activation('softmax'))
    return self.model

  def apply_softmax(self, logits):
    return tf.nn.softmax(logits)

  def train(self, train_data):

    for step, (img_batch, lbl_batch) in enumerate(train_data):

      with tf.GradientTape() as tape:
          logits = self.model(img_batch)
#          logits = self.apply_softmax(logits)
          xent = self.loss_fn(lbl_batch, logits)

      varis = self.model.trainable_variables
      grads = tape.gradient(xent, varis)

      self.opt.apply_gradients(zip(grads, varis))
      
      self.train_acc_metric(lbl_batch, logits)
      
      if tf.equal(step % 100, 0): #not step % 500:
        acc = self.train_acc_metric.result()
        tf.print("Loss: {} Accuracy: {}".format(xent, acc))
        self.train_acc_metric.reset_states()

  def train2(self, train_imgs, train_lbls, val_imgs, val_lbls):
    self.model.compile(
      optimizer='adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['accuracy']
      )

    self.model.fit(train_imgs, train_lbls, validation_data= (val_imgs, val_lbls), epochs=20)

  def test(self,test_data, temp=None):
    for img_batch, lbl_batch in test_data:
      logits = self.model(img_batch)
      if not temp is None:
        logits = tf.math.divide(logits, temp)
      pred = self.apply_softmax(logits)
      self.test_acc_metric(lbl_batch, pred)
    tf.print("Test acc: {}".format(self.test_acc_metric.result()))

  def save_model(self, path):
    self.model.save(path)

  def load_model(self, path):
    self.model = tf.keras.models.load_model(path)
    return self.model

  def get_max_pred_value(self, data, temp=None):
    logits = self.model(data)
    if not temp is None:
      logits = tf.math.divide(logits, temp)
    pred = self.apply_softmax(logits)
    return np.max(pred)


In [ ]:
pickle_in = open("mnist_data","rb")
train_images, train_labels, val_images, val_labels, test_imgs, test_lbls = pickle.load(pickle_in)

In [ ]:
def separate_class(class_val, train_images, train_labels, val_images, val_labels):
  new_train_images = []
  new_train_labels = []
  for ind, im in enumerate(train_images):
    if train_labels[ind] != class_val:
      new_train_images.append(im)
      new_train_labels.append(train_labels[ind])

  new_train_images = np.asarray(new_train_images)
  new_train_labels = np.asarray(new_train_labels)

  new_val_images = []
  new_val_labels = []
  for ind, im in enumerate(val_images):
    if val_labels[ind] != class_val:
      new_val_images.append(im)
      new_val_labels.append(val_labels[ind])

  new_val_images = np.asarray(new_val_images)
  new_val_labels = np.asarray(new_val_labels)

  return new_train_images, new_train_labels, new_val_images, new_val_labels

In [ ]:
def get_train_data(leave_out_class, train_images, train_labels):
  class_train_images = []
  class_train_labels = []
  mapping = {}
  
  labels = set(train_labels)
  labels.remove(leave_out_class)
  labels = sorted(labels)
  for i,j in enumerate(labels):
    mapping[j] = i
  print(mapping)

  for ind, im in enumerate(train_images):
    if train_labels[ind] != leave_out_class:
      class_train_images.append(im)
      class_train_labels.append(mapping[train_labels[ind]]) #(train_labels[ind])


  class_train_images = np.asarray(class_train_images)
  class_train_labels = np.asarray(class_train_labels)

  print(set(class_train_labels))
  print(class_train_labels.shape)

  train_data = tf.data.Dataset.from_tensor_slices((class_train_images, class_train_labels))

  train_data = train_data.shuffle(class_train_images.shape[0])
  train_data = train_data.batch(128)
  train_data = train_data.repeat(20)

  return class_train_images, class_train_labels # train_data

In [ ]:
def temp_cal(y_pred, val_labels):

  temp = tf.Variable(initial_value=1.0, trainable=True, dtype=tf.float32) 

  def compute_loss():
      y_pred_model_w_temp = tf.math.divide(y_pred, temp)
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(\
                                  tf.convert_to_tensor(keras.utils.to_categorical(val_labels)), y_pred_model_w_temp))
      return loss

  optimizer = tf.optimizers.Adam(learning_rate=0.01)

  print('Temperature Initial value: {}'.format(temp.numpy()))

  for i in range(300):
      opts = optimizer.minimize(compute_loss, var_list=[temp])


  print('Temperature Final value: {}'.format(temp.numpy()))

  return temp

In [ ]:
def temp_scaling(y_pred, temp):
  return tf.math.divide(y_pred, temp)

In [ ]:
def get_mapping(leave_out_class, anomaly_class):
  mapping = {}

  labels = set(train_labels)
  labels.remove(anomaly_class)
  labels.remove(leave_out_class)
  labels = sorted(labels)

  for i,j in enumerate(labels):
    mapping[i] = j

  return mapping


# 0 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(0, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='0anomaly:classifier0')
model1 = Model(name='0anomaly:classifier1')
model2 = Model(name='0anomaly:classifier2')
model3 = Model(name='0anomaly:classifier3')
model4 = Model(name='0anomaly:classifier4')
model5 = Model(name='0anomaly:classifier5')
model6 = Model(name='0anomaly:classifier6')
model7 = Model(name='0anomaly:classifier7')
model8 = Model(name='0anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/0anomaly-classifier0")
class_1 = model1.load_model("final_models/0anomaly-classifier1")
class_2 = model2.load_model("final_models/0anomaly-classifier2")
class_3 = model3.load_model("final_models/0anomaly-classifier3")
class_4 = model4.load_model("final_models/0anomaly-classifier4")
class_5 = model5.load_model("final_models/0anomaly-classifier5")
class_6 = model6.load_model("final_models/0anomaly-classifier6")
class_7 = model7.load_model("final_models/0anomaly-classifier7")
class_8 = model8.load_model("final_models/0anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35', 'dense_37', 'dense_39', 'dense_41']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.2917109, 1.7892319, 2.1682718, 2.1920147, 2.3539624, 1.867185, 2.2140534, 1.8692174, 2.201385]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_0an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_0an.append((pred, label))

10825it [13:31, 13.34it/s]


In [ ]:
len(softmax_data_0an)

97425

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_0an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86600, 10825)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21648

In [ ]:
idata

(<tf.Tensor: shape=(1, 8), dtype=float32, numpy=
 array([[0.00427151, 0.8727716 , 0.00139973, 0.08199269, 0.00370487,
         0.0014508 , 0.01937814, 0.01503068]], dtype=float32)>, 1)

In [ ]:
input[0], labels[0], len(input)

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([8.3075048e-07, 5.9338680e-07, 3.6874508e-06, 9.2867964e-08,
        4.6701287e-09, 9.9999249e-01, 5.7111261e-08, 2.1998203e-06],
       dtype=float32)>, 1, 21650)

In [ ]:
clf = make_pipeline(SVC(gamma='auto'))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_0an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 0:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_0an.append((output, label))

10000it [12:29, 13.35it/s]


In [ ]:
sm_all_0an[0]

In [ ]:
pickle_out = open("sm_all_wt_m2_0an.pickle","wb")
pickle.dump([softmax_data_0an, sm_all_0an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_0an.pickle","rb")
softmax_data_0an, sm_all_0an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_0an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    output_nb.append(gnb.predict(pred))
#    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)


  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(test_imgs)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(test_imgs)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9447
Accuracy of determining ID data:  0.973170731707317
Accuracy of determining OOD data:  0.6826530612244898
-------NN-----------
Total Accuracy:  0.3821
Accuracy of determining ID data:  0.33580931263858094
Accuracy of determining OOD data:  0.8081632653061225


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.8279118964659035

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5719862889723518

# 1 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(1, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='1anomaly:classifier0')
model1 = Model(name='1anomaly:classifier1')
model2 = Model(name='1anomaly:classifier2')
model3 = Model(name='1anomaly:classifier3')
model4 = Model(name='1anomaly:classifier4')
model5 = Model(name='1anomaly:classifier5')
model6 = Model(name='1anomaly:classifier6')
model7 = Model(name='1anomaly:classifier7')
model8 = Model(name='1anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/1anomaly-classifier0")
class_1 = model1.load_model("final_models/1anomaly-classifier1")
class_2 = model2.load_model("final_models/1anomaly-classifier2")
class_3 = model3.load_model("final_models/1anomaly-classifier3")
class_4 = model4.load_model("final_models/1anomaly-classifier4")
class_5 = model5.load_model("final_models/1anomaly-classifier5")
class_6 = model6.load_model("final_models/1anomaly-classifier6")
class_7 = model7.load_model("final_models/1anomaly-classifier7")
class_8 = model8.load_model("final_models/1anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_61', 'dense_63', 'dense_65', 'dense_67', 'dense_69', 'dense_71', 'dense_73', 'dense_75', 'dense_19']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.0609157, 2.3180223, 2.2263615, 2.5107794, 2.1667995, 2.330876, 1.9582385, 2.1498156, 2.477948]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 2, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_1an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_1an.append((pred, label))

10678it [20:44,  8.58it/s]


In [ ]:
len(softmax_data_1an)

96102

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_1an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(85424, 10678)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21356

In [ ]:
input[0], labels[0], len(input)

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([6.7688677e-07, 3.0091948e-07, 5.4442139e-06, 5.2381586e-07,
        4.8233282e-08, 9.9993050e-01, 5.7272548e-08, 6.2568732e-05],
       dtype=float32)>, 1, 21356)

In [ ]:
input = []
labels = []
for data in softmax_data_1an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
input[0]

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([6.7688677e-07, 3.0091948e-07, 5.4442139e-06, 5.2381586e-07,
       4.8233282e-08, 9.9993050e-01, 5.7272548e-08, 6.2568732e-05],
      dtype=float32)>

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_1an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 1:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_1an.append((output, label))

10000it [14:26, 11.54it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_1an.pickle","wb")
pickle.dump([softmax_data_1an, sm_all_1an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_1an.pickle","rb")
softmax_data_1an, sm_all_1an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_1an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_1an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_1an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.8739
Accuracy of determining ID data:  0.8589960518894529
Accuracy of determining OOD data:  0.9903083700440528
-------NN-----------
Total Accuracy:  0.8865
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.9246522109667529

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 2 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(2, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='2anomaly:classifier0')
model1 = Model(name='2anomaly:classifier1')
model2 = Model(name='2anomaly:classifier2')
model3 = Model(name='2anomaly:classifier3')
model4 = Model(name='2anomaly:classifier4')
model5 = Model(name='2anomaly:classifier5')
model6 = Model(name='2anomaly:classifier6')
model7 = Model(name='2anomaly:classifier7')
model8 = Model(name='2anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/2anomaly-classifier0")
class_1 = model1.load_model("final_models/2anomaly-classifier1")
class_2 = model2.load_model("final_models/2anomaly-classifier2")
class_3 = model3.load_model("final_models/2anomaly-classifier3")
class_4 = model4.load_model("final_models/2anomaly-classifier4")
class_5 = model5.load_model("final_models/2anomaly-classifier5")
class_6 = model6.load_model("final_models/2anomaly-classifier6")
class_7 = model7.load_model("final_models/2anomaly-classifier7")
class_8 = model8.load_model("final_models/2anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.246656, 2.2101953, 2.2647355, 2.2229295, 2.1457264, 2.1438227, 2.1131978, 2.0234013, 2.4829793]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 3, 4, 5, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_2an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_2an.append((pred, label))

10826it [15:38, 11.54it/s]


In [ ]:
len(softmax_data_2an)

97434

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_2an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86608, 10826)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21652

In [ ]:
input = []
labels = []
for data in softmax_data_2an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_2an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 2:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_2an.append((output, label))

10000it [13:43, 12.14it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_2an.pickle","wb")
pickle.dump([softmax_data_2an, sm_all_2an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_2an.pickle","rb")
softmax_data_2an, sm_all_2an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_2an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_2an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_2an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9358
Accuracy of determining ID data:  0.965321141837645
Accuracy of determining OOD data:  0.6792635658914729
-------NN-----------
Total Accuracy:  0.8968
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.822292353864559

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 3 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(3, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='3anomaly:classifier0')
model1 = Model(name='3anomaly:classifier1')
model2 = Model(name='3anomaly:classifier2')
model3 = Model(name='3anomaly:classifier3')
model4 = Model(name='3anomaly:classifier4')
model5 = Model(name='3anomaly:classifier5')
model6 = Model(name='3anomaly:classifier6')
model7 = Model(name='3anomaly:classifier7')
model8 = Model(name='3anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/3anomaly-classifier0")
class_1 = model1.load_model("final_models/3anomaly-classifier1")
class_2 = model2.load_model("final_models/3anomaly-classifier2")
class_3 = model3.load_model("final_models/3anomaly-classifier3")
class_4 = model4.load_model("final_models/3anomaly-classifier4")
class_5 = model5.load_model("final_models/3anomaly-classifier5")
class_6 = model6.load_model("final_models/3anomaly-classifier6")
class_7 = model7.load_model("final_models/3anomaly-classifier7")
class_8 = model8.load_model("final_models/3anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_19', 'dense_21', 'dense_23', 'dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [1.9315488, 2.0040166, 1.7066195, 2.314881, 2.081745, 1.9893513, 2.4744115, 2.3960419, 2.1575763]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 4, 5, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_3an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_3an.append((pred, label))

10781it [15:01, 11.96it/s]


In [ ]:
len(softmax_data_3an)

97029

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_3an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86248, 10781)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21560

In [ ]:
input = []
labels = []
for data in softmax_data_3an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_3an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 3:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_3an.append((output, label))

10000it [14:06, 11.82it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_3an.pickle","wb")
pickle.dump([softmax_data_3an, sm_all_3an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_3an.pickle","rb")
softmax_data_3an, sm_all_3an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_3an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_3an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_3an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9304
Accuracy of determining ID data:  0.9701890989988876
Accuracy of determining OOD data:  0.5762376237623762
-------NN-----------
Total Accuracy:  0.899
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.773213361380632

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 4 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(4, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='4anomaly:classifier0')
model1 = Model(name='4anomaly:classifier1')
model2 = Model(name='4anomaly:classifier2')
model3 = Model(name='4anomaly:classifier3')
model4 = Model(name='4anomaly:classifier4')
model5 = Model(name='4anomaly:classifier5')
model6 = Model(name='4anomaly:classifier6')
model7 = Model(name='4anomaly:classifier7')
model8 = Model(name='4anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/4anomaly-classifier0")
class_1 = model1.load_model("final_models/4anomaly-classifier1")
class_2 = model2.load_model("final_models/4anomaly-classifier2")
class_3 = model3.load_model("final_models/4anomaly-classifier3")
class_4 = model4.load_model("final_models/4anomaly-classifier4")
class_5 = model5.load_model("final_models/4anomaly-classifier5")
class_6 = model6.load_model("final_models/4anomaly-classifier6")
class_7 = model7.load_model("final_models/4anomaly-classifier7")
class_8 = model8.load_model("final_models/4anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.1738567, 2.3370507, 2.1365163, 2.441992, 2.217251, 2.4548638, 2.1809883, 2.343067, 2.4618614]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 5, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_4an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_4an.append((pred, label))

10824it [15:19, 11.77it/s]


In [ ]:
len(softmax_data_4an)

97416

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_4an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86592, 10824)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21560

In [ ]:
input = []
labels = []
for data in softmax_data_4an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_4an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 4:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_4an.append((output, label))

10000it [14:01, 11.88it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_4an.pickle","wb")
pickle.dump([softmax_data_4an, sm_all_4an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_4an.pickle","rb")
softmax_data_4an, sm_all_4an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_4an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_4an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_4an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9168
Accuracy of determining ID data:  0.961964958970947
Accuracy of determining OOD data:  0.5020366598778004
-------NN-----------
Total Accuracy:  0.9018
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.7320008094243736

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 5 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(5, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='5anomaly:classifier0')
model1 = Model(name='5anomaly:classifier1')
model2 = Model(name='5anomaly:classifier2')
model3 = Model(name='5anomaly:classifier3')
model4 = Model(name='5anomaly:classifier4')
model5 = Model(name='5anomaly:classifier5')
model6 = Model(name='5anomaly:classifier6')
model7 = Model(name='5anomaly:classifier7')
model8 = Model(name='5anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/5anomaly-classifier0")
class_1 = model1.load_model("final_models/5anomaly-classifier1")
class_2 = model2.load_model("final_models/5anomaly-classifier2")
class_3 = model3.load_model("final_models/5anomaly-classifier3")
class_4 = model4.load_model("final_models/5anomaly-classifier4")
class_5 = model5.load_model("final_models/5anomaly-classifier5")
class_6 = model6.load_model("final_models/5anomaly-classifier6")
class_7 = model7.load_model("final_models/5anomaly-classifier7")
class_8 = model8.load_model("final_models/5anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_93', 'dense_95', 'dense_97', 'dense_99', 'dense_101', 'dense_103', 'dense_105', 'dense_107', 'dense_19']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.056544, 2.1163602, 2.3622563, 2.3906677, 2.2536275, 2.3036711, 2.5086586, 2.2106051, 2.266882]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 4, 6, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_5an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_5an.append((pred, label))

10896it [15:30, 11.71it/s]


In [ ]:
len(softmax_data_5an)

98064

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_5an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(87168, 10896)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21792

In [ ]:
input = []
labels = []
for data in softmax_data_5an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_5an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 5:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_5an.append((output, label))

10000it [14:05, 11.83it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_5an.pickle","wb")
pickle.dump([softmax_data_5an, sm_all_5an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_5an.pickle","rb")
softmax_data_5an, sm_all_5an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_5an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_5an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_5an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9478
Accuracy of determining ID data:  0.9687088274044796
Accuracy of determining OOD data:  0.734304932735426
-------NN-----------
Total Accuracy:  0.9108
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.8515068800699528

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 6 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(6, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='6anomaly:classifier0')
model1 = Model(name='6anomaly:classifier1')
model2 = Model(name='6anomaly:classifier2')
model3 = Model(name='6anomaly:classifier3')
model4 = Model(name='6anomaly:classifier4')
model5 = Model(name='6anomaly:classifier5')
model6 = Model(name='6anomaly:classifier6')
model7 = Model(name='6anomaly:classifier7')
model8 = Model(name='6anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/6anomaly-classifier0")
class_1 = model1.load_model("final_models/6anomaly-classifier1")
class_2 = model2.load_model("final_models/6anomaly-classifier2")
class_3 = model3.load_model("final_models/6anomaly-classifier3")
class_4 = model4.load_model("final_models/6anomaly-classifier4")
class_5 = model5.load_model("final_models/6anomaly-classifier5")
class_6 = model6.load_model("final_models/6anomaly-classifier6")
class_7 = model7.load_model("final_models/6anomaly-classifier7")
class_8 = model8.load_model("final_models/6anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53', 'dense_55']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.3399413, 2.323194, 2.275749, 2.1671793, 2.4774106, 2.007354, 2.019063, 2.4026806, 2.4807267]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 4, 5, 7, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_6an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_6an.append((pred, label))

10823it [15:11, 11.87it/s]


In [ ]:
len(softmax_data_6an)

97407

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_6an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86584, 10823)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21644

In [ ]:
input = []
labels = []
for data in softmax_data_6an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_6an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 6:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_6an.append((output, label))

10000it [14:01, 11.88it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_6an.pickle","wb")
pickle.dump([softmax_data_6an, sm_all_6an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_6an.pickle","rb")
softmax_data_6an, sm_all_6an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_6an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_6an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_6an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9255
Accuracy of determining ID data:  0.967042689670427
Accuracy of determining OOD data:  0.5334029227557411
-------NN-----------
Total Accuracy:  0.8334
Accuracy of determining ID data:  0.9207033842070338
Accuracy of determining OOD data:  0.009394572025052192


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.750222806213084

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.465048978116043

# 7 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(7, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='7anomaly:classifier0')
model1 = Model(name='7anomaly:classifier1')
model2 = Model(name='7anomaly:classifier2')
model3 = Model(name='7anomaly:classifier3')
model4 = Model(name='7anomaly:classifier4')
model5 = Model(name='7anomaly:classifier5')
model6 = Model(name='7anomaly:classifier6')
model7 = Model(name='7anomaly:classifier7')
model8 = Model(name='7anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/7anomaly-classifier0")
class_1 = model1.load_model("final_models/7anomaly-classifier1")
class_2 = model2.load_model("final_models/7anomaly-classifier2")
class_3 = model3.load_model("final_models/7anomaly-classifier3")
class_4 = model4.load_model("final_models/7anomaly-classifier4")
class_5 = model5.load_model("final_models/7anomaly-classifier5")
class_6 = model6.load_model("final_models/7anomaly-classifier6")
class_7 = model7.load_model("final_models/7anomaly-classifier7")
class_8 = model8.load_model("final_models/7anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_37', 'dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.1840894, 2.3121402, 1.8578718, 2.0179794, 2.3343472, 2.30917, 2.2927048, 2.3133302, 2.1407003]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 4, 5, 6, 8, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_7an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_7an.append((pred, label))

10701it [14:52, 11.99it/s]


In [ ]:
len(softmax_data_7an)

96309

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_7an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(85608, 10701)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21400

In [ ]:
input = []
labels = []
for data in softmax_data_7an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_7an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 7:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_7an.append((output, label))

10000it [14:12, 11.74it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_7an.pickle","wb")
pickle.dump([softmax_data_7an, sm_all_7an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_3an.pickle","rb")
softmax_data_3an, sm_all_3an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_7an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_7an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_7an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9604
Accuracy of determining ID data:  0.9712438698172091
Accuracy of determining OOD data:  0.8657587548638133
-------NN-----------
Total Accuracy:  0.8972
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.9185013123405112

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5

# 8 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(8, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='8anomaly:classifier0')
model1 = Model(name='8anomaly:classifier1')
model2 = Model(name='8anomaly:classifier2')
model3 = Model(name='8anomaly:classifier3')
model4 = Model(name='8anomaly:classifier4')
model5 = Model(name='8anomaly:classifier5')
model6 = Model(name='8anomaly:classifier6')
model7 = Model(name='8anomaly:classifier7')
model8 = Model(name='8anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/8anomaly-classifier0")
class_1 = model1.load_model("final_models/8anomaly-classifier1")
class_2 = model2.load_model("final_models/8anomaly-classifier2")
class_3 = model3.load_model("final_models/8anomaly-classifier3")
class_4 = model4.load_model("final_models/8anomaly-classifier4")
class_5 = model5.load_model("final_models/8anomaly-classifier5")
class_6 = model6.load_model("final_models/8anomaly-classifier6")
class_7 = model7.load_model("final_models/8anomaly-classifier7")
class_8 = model8.load_model("final_models/8anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_19', 'dense_21', 'dense_23', 'dense_25', 'dense_27', 'dense_29', 'dense_31', 'dense_33', 'dense_35']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.2895179, 2.1803794, 1.9876496, 1.944436, 2.4403906, 1.9343997, 2.2423983, 2.2220883, 2.2203646]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 4, 5, 6, 7, 9]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_8an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_8an.append((pred, label))

10840it [15:25, 11.72it/s]


In [ ]:
len(softmax_data_8an)

97560

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_8an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86720, 10840)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21680

In [ ]:
input = []
labels = []
for data in softmax_data_8an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_8an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 8:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_8an.append((output, label))

10000it [14:09, 11.77it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_8an.pickle","wb")
pickle.dump([softmax_data_8an, sm_all_8an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_8an.pickle","rb")
softmax_data_8an, sm_all_8an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_8an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_8an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_8an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.944
Accuracy of determining ID data:  0.964103700421006
Accuracy of determining OOD data:  0.757700205338809
-------NN-----------
Total Accuracy:  0.8126
Accuracy of determining ID data:  0.890206071349435
Accuracy of determining OOD data:  0.09342915811088295


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.8609019528799075

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.49181761473015895

# 9 as anomaly

In [ ]:
new_train_images, new_train_labels, new_val_images, new_val_labels = separate_class(9, train_images, train_labels, val_images, val_labels)

In [ ]:
model0 = Model(name='9anomaly:classifier0')
model1 = Model(name='9anomaly:classifier1')
model2 = Model(name='9anomaly:classifier2')
model3 = Model(name='9anomaly:classifier3')
model4 = Model(name='9anomaly:classifier4')
model5 = Model(name='9anomaly:classifier5')
model6 = Model(name='9anomaly:classifier6')
model7 = Model(name='9anomaly:classifier7')
model8 = Model(name='9anomaly:classifier8')

In [ ]:
class_0 = model0.build()
class_1 = model1.build()
class_2 = model2.build()
class_3 = model3.build()
class_4 = model4.build()
class_5 = model5.build()
class_6 = model6.build()
class_7 = model7.build()
class_8 = model8.build()

In [ ]:
class_0 = model0.load_model("final_models/9anomaly-classifier0")
class_1 = model1.load_model("final_models/9anomaly-classifier1")
class_2 = model2.load_model("final_models/9anomaly-classifier2")
class_3 = model3.load_model("final_models/9anomaly-classifier3")
class_4 = model4.load_model("final_models/9anomaly-classifier4")
class_5 = model5.load_model("final_models/9anomaly-classifier5")
class_6 = model6.load_model("final_models/9anomaly-classifier6")
class_7 = model7.load_model("final_models/9anomaly-classifier7")
class_8 = model8.load_model("final_models/9anomaly-classifier8")

In [ ]:
layer_name_list = []

layer_name_list.append(class_0.layers[-2:][0].name)
layer_name_list.append(class_1.layers[-2:][0].name)
layer_name_list.append(class_2.layers[-2:][0].name)
layer_name_list.append(class_3.layers[-2:][0].name)
layer_name_list.append(class_4.layers[-2:][0].name)
layer_name_list.append(class_5.layers[-2:][0].name)
layer_name_list.append(class_6.layers[-2:][0].name)
layer_name_list.append(class_7.layers[-2:][0].name)
layer_name_list.append(class_8.layers[-2:][0].name)

print(layer_name_list)

['dense_37', 'dense_39', 'dense_41', 'dense_43', 'dense_45', 'dense_47', 'dense_49', 'dense_51', 'dense_53']


In [ ]:
from keras import Model
model_dict = {}
model_output0 = class_0.get_layer(layer_name_list[0]).output
m0 = Model(inputs=class_0.input, outputs=model_output0)
model_dict[0] = m0

model_output1 = class_1.get_layer(layer_name_list[1]).output
m1 = Model(inputs=class_1.input, outputs=model_output1)
model_dict[1] = m1

model_output2 = class_2.get_layer(layer_name_list[2]).output
m2 = Model(inputs=class_2.input, outputs=model_output2)
model_dict[2] = m2

model_output3 = class_3.get_layer(layer_name_list[3]).output
m3 = Model(inputs=class_3.input, outputs=model_output3)
model_dict[3] = m3

model_output4 = class_4.get_layer(layer_name_list[4]).output
m4 = Model(inputs=class_4.input, outputs=model_output4)
model_dict[4] = m4

model_output5 = class_5.get_layer(layer_name_list[5]).output
m5 = Model(inputs=class_5.input, outputs=model_output5)
model_dict[5] = m5

model_output6 = class_6.get_layer(layer_name_list[6]).output
m6 = Model(inputs=class_6.input, outputs=model_output6)
model_dict[6] = m6

model_output7 = class_7.get_layer(layer_name_list[7]).output
m7 = Model(inputs=class_7.input, outputs=model_output7)
model_dict[7] = m7

model_output8 = class_8.get_layer(layer_name_list[8]).output
m8 = Model(inputs=class_8.input, outputs=model_output8)
model_dict[8] = m8

In [ ]:
temp_val = [2.3985262, 2.319251, 2.2143555, 1.9258661, 2.2611945, 2.2223954, 2.3583956, 2.2993236, 2.0704982]

In [ ]:
leave_out_list = sorted(set(new_val_labels))

In [ ]:
leave_out_list

[0, 1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
ood = 0
ind = 1

In [ ]:
from tqdm import tqdm
softmax_data_9an = []

for img, lbl in tqdm(zip(new_val_images,new_val_labels)):
  img = img.reshape([-1, 28, 28, 1])
  for i in range(9):
    if lbl == leave_out_list[i]:
      label = ood
    else:
      label = ind
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    softmax_data_9an.append((pred, label))

10806it [15:25, 11.67it/s]


In [ ]:
len(softmax_data_9an)

97254

In [ ]:
ind_data = []
ood_data = []
for data in softmax_data_9an:
  if data[1] == ind:
    ind_data.append(data)
  else:
    ood_data.append(data)

In [ ]:
len(ind_data), len(ood_data)

(86448, 10806)

In [ ]:
input = []
labels = []
for i in range(0, len(ood_data)-1, 2):
  input.append(ind_data[i][0][0])
  input.append(ind_data[i+1][0][0])
  labels.append(ind_data[i][1])
  labels.append(ind_data[i+1][1])
  input.append(ood_data[i][0][0])
  input.append(ood_data[i+1][0][0])
  labels.append(ood_data[i][1])
  labels.append(ood_data[i][1])

#for idata, odata in zip(ind_data[:len(ood_data)],ood_data):
#  input.append(idata[0][0])
#  input.append(odata[0][0])
#  labels.append(idata[1])
#  labels.append(odata[1])  

In [ ]:
len(labels)

21612

In [ ]:
input = []
labels = []
for data in softmax_data_9an:
  input.append(data[0][0])
  labels.append(data[1])

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_wts = compute_class_weight('balanced', np.unique(labels), labels)
class_wts ={0:class_wts[0], 1:class_wts[1]}
class_wts

{0: 4.5, 1: 0.5625}

In [ ]:
clf = make_pipeline(SVC(gamma='auto', class_weight=class_wts))
clf.fit(input, labels)

Pipeline(memory=None,
         steps=[('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200,
                     class_weight={0: 4.5, 1: 0.5625}, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [ ]:
gnb = GaussianNB()
gnb.fit(input, labels)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
clfp = Perceptron(tol=1e-3, random_state=0)
clfp.fit(input, labels)

Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=1000, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=0, shuffle=True, tol=0.001,
           validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
sm_all_9an = []
for data, lbl in tqdm(zip(test_imgs, test_lbls)):
  img = data.reshape([-1, 28, 28, 1])
  if lbl == 9:
    label = ood
  else:
    label = ind

  output = []
  for i in range(9):  
    logits = model_dict[i](img)
    logits = tf.math.divide(logits, temp_val[i])
    pred = tf.nn.softmax(logits)
    output.append(pred)
  
  sm_all_9an.append((output, label))

10000it [14:01, 11.89it/s]


In [ ]:
pickle_out = open("sm_all_wt_m2_9an.pickle","wb")
pickle.dump([softmax_data_9an, sm_all_9an], pickle_out)
pickle_out.close()

In [ ]:
pickle_in = open("sm_all_wt_m2_9an.pickle","rb")
softmax_data_9an, sm_all_9an = pickle.load(pickle_in)

In [ ]:
accuracy_svm = 0.0
accuracy_nb = 0.0
acc_in_svm = 0.0
acc_in_nb = 0.0
acc_ood_svm = 0.0
acc_ood_nb = 0.0
count_in = 0
count_ood = 0
y_pred_svm = []
y_pred_nb = []
y_true = []

for data, lbl in sm_all_9an:
  output_svm = []
  output_nb = []
  if lbl == ood:
    count_ood += 1
  else:
    count_in += 1

  for pred in data:
    output_svm.append(clf.predict(pred))
    #output_nb.append(gnb.predict(pred))
    output_nb.append(clfp.predict(pred))

  y_true.append(lbl)

  values0, counts0 = np.unique(output_svm, return_counts=True)
  y_pred0 = values0[np.argmax(counts0)]
  y_pred_svm.append(y_pred0)

  values1, counts1 = np.unique(output_nb, return_counts=True)
  y_pred1 = values1[np.argmax(counts1)]
  y_pred_nb.append(y_pred1)

  if y_pred0 == lbl:
    accuracy_svm += 1
    if lbl == ind:
      acc_in_svm += 1
    else:
      acc_ood_svm += 1

  if y_pred1 == lbl:
    accuracy_nb += 1
    if lbl == ind:
      acc_in_nb += 1
    else:
      acc_ood_nb += 1

print('-------SVM----------')
print("Total Accuracy: ", accuracy_svm/len(sm_all_9an)) 
print("Accuracy of determining ID data: ", acc_in_svm/count_in)
print("Accuracy of determining OOD data: ", acc_ood_svm/count_ood)

print('-------NN-----------')
print("Total Accuracy: ", accuracy_nb/len(sm_all_9an)) 
print("Accuracy of determining ID data: ", acc_in_nb/count_in)
print("Accuracy of determining OOD data: ", acc_ood_nb/count_ood)

-------SVM----------
Total Accuracy:  0.9435
Accuracy of determining ID data:  0.9706373039706373
Accuracy of determining OOD data:  0.7016848364717542
-------NN-----------
Total Accuracy:  0.8991
Accuracy of determining ID data:  1.0
Accuracy of determining OOD data:  0.0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true, y_pred_svm)

0.8361610702211959

In [ ]:
roc_auc_score(y_true, y_pred_nb)

0.5